In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns' , 26)

In [3]:
df = pd.read_csv(r"C:\Users\ahmed\Downloads\final_internship_data.csv")
df

User ID          User Name       Driver Name Car Condition Weather  \
0       KHVrEVlD     Kimberly Adams        Amy Butler     Very Good   windy   
1       lPxIuEri       Justin Tapia  Hannah Zimmerman     Excellent  cloudy   
2       gsVN8JLS    Elizabeth Lopez    Amanda Jackson           Bad  stormy   
3       9I7kWFgd      Steven Wilson          Amy Horn     Very Good  stormy   
4       8QN5ZaGN  Alexander Andrews  Cassandra Larson           Bad  stormy   
...          ...                ...               ...           ...     ...   
499995  er7Luy6J       Morgan Smith   Gabriel Coleman           Bad   rainy   
499996  FWDZu9NA     Dillon Jackson    Larry Richards     Very Good   rainy   
499997  QZIMcpLZ  Michael Rodriguez    Lonnie Santana           Bad   rainy   
499998  AD72Uwmn       Deborah Soto      Emily Rivera          Good  stormy   
499999  qGKn4Um5         Eric Jones        Julie King     Very Good   sunny   

        Traffic Condition                            key  fare_amount  \
0       Congested Traffic    2009-06-15 17:26:21.0000001          4.5   
1            Flow Traffic    2010-01-05 16:52:16.0000002         16.9   
2       Congested Traffic   2011-08-18 00:35:00.00000049          5.7   
3            Flow Traffic    2012-04-21 04:30:42.0000001          7.7   
4       Congested Traffic  2010-03-09 07:51:00.000000135          5.3   
...                   ...                            ...          ...   
499995      Dense Traffic    2015-05-07 18:45:12.0000004          7.0   
499996       Flow Traffic    2010-09-13 12:11:34.0000004         13.7   
499997       Flow Traffic    2014-08-25 00:22:20.0000001         25.0   
499998  Congested Traffic    2015-01-12 12:17:32.0000001          6.5   
499999       Flow Traffic    2010-04-12 23:46:58.0000002          4.9   

            pickup_datetime  pickup_longitude  pickup_latitude  \
0       2009-06-15 17:26:21         -1.288826         0.710721   
1       2010-01-05 16:52:16         -1.291824         0.710546   
2       2011-08-18 00:35:00         -1.291242         0.711418   
3       2012-04-21 04:30:42         -1.291319         0.710927   
4       2010-03-09 07:51:00         -1.290987         0.711536   
...                     ...               ...              ...   
499995  2015-05-07 18:45:12         -1.291173         0.711514   
499996  2010-09-13 12:11:34         -1.291595         0.710670   
499997  2014-08-25 00:22:20         -1.291262         0.710796   
499998  2015-01-12 12:17:32         -1.291101         0.711335   
499999  2010-04-12 23:46:58         -1.291312         0.710736   

        dropoff_longitude  dropoff_latitude  passenger_count  hour  day  \
0               -1.288779          0.710563                1    17   15   
1               -1.291182          0.711780                1    16    5   
2               -1.291391          0.711231                2     0   18   
3               -1.291396          0.711363                1     4   21   
4               -1.290787          0.711811                1     7    9   
...                   ...               ...              ...   ...  ...   
499995          -1.290951          0.711371                1    18    7   
499996          -1.291708          0.710540                1    12   13   
499997          -1.289737          0.710366                1     0   25   
499998          -1.291023          0.711480                6    12   12   
499999          -1.291243          0.710582                1    23   12   

        month  weekday  year   jfk_dist   ewr_dist   lga_dist   sol_dist  \
0           6        0  2009  20.265840  55.176046  14.342611  34.543548   
1           1        1  2010  44.667679  31.832358  23.130775  15.125872   
2           8        3  2011  43.597686  33.712082  19.865289  17.722624   
3           4        5  2012  42.642965  32.556289  21.063132  15.738963   
4           3        1  2010  43.329953  39.406828  15.219339  23.732406   
...       ...      ...   ...     

In [4]:
df = df.dropna()

In [5]:
def drop_outliers(df):
    """
    Remove outliers from numerical columns in the DataFrame using the IQR method.

    """
    # Make a copy of the DataFrame to avoid modifying the original
    df_clean = df.copy()

    # Iterate over each column in the DataFrame
    for col in df_clean.columns:
        if df_clean[col].dtype in ['float64', 'int64', 'int32']:
            # Calculate Q1 (25th percentile) and Q3 (75th percentile)
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)

            # Calculate the Interquartile Range (IQR)
            IQR = Q3 - Q1

            # Define lower and upper bounds for outliers
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Drop rows with outliers in the current column
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]

    return df_clean

In [6]:
df_dropped_outliers = drop_outliers(df)

In [7]:
df_dropped_outliers = df_dropped_outliers.drop(columns=['User ID' , 'User Name' , 'Driver Name' , 'key' , 'pickup_datetime' ])

# Data encoding using **Label Encoder**

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
# List of categorical columns to be encoded
categorical_columns = ['Car Condition' , 'Weather' , 'Traffic Condition']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column
for col in categorical_columns:
    df_dropped_outliers[col] = label_encoder.fit_transform(df_dropped_outliers[col])



# Feature selection using **ANova**

In [10]:
x = df_dropped_outliers.drop(columns = ['fare_amount'])
y = df_dropped_outliers['fare_amount']

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import StandardScaler

In [12]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

In [13]:
# Define a function for feature selection
def feature_selection(X, y, X1):
    # Perform feature selection using mutual information
    selector = SelectKBest(mutual_info_regression, k=7)  # Select top 7 features
    X_new = selector.fit_transform(X, y)
    selected_feature_indices = selector.get_support(indices=True)
    selected_feature_names = X1.columns[selected_feature_indices]
    print("Selected feature names:", selected_feature_names)
    print("The number of features:", len(selected_feature_names))
    return X_new, y

In [14]:
# Create a DataFrame with columns for easy reference
X1 = pd.DataFrame(X_scaled, columns=df_dropped_outliers.drop(columns=['fare_amount']).columns)

In [15]:
# Perform feature selection
X_selected, y_selected = feature_selection(X_scaled, y, X1)


Selected feature names: Index(['dropoff_latitude', 'year', 'ewr_dist', 'lga_dist', 'nyc_dist',
       'distance', 'bearing'],
      dtype='object')
The number of features: 7


In [16]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_selected, test_size=0.2, random_state=42)

# XG Boost

In [17]:
import xgboost as xgb

In [18]:
# Create XGBoost DMatrix
train_data = xgb.DMatrix(X_train, label=y_train)
test_data = xgb.DMatrix(X_test, label=y_test)


In [19]:
# Define initial XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'seed': 42
}

In [20]:
# Train the XGBoost model
model = xgb.train(params, train_data, num_boost_round=100, early_stopping_rounds=10, evals=[(test_data, 'eval')])

[0]	eval-rmse:2.80493
[1]	eval-rmse:2.42422
[2]	eval-rmse:2.20311
[3]	eval-rmse:2.07676
[4]	eval-rmse:2.00435
[5]	eval-rmse:1.96183
[6]	eval-rmse:1.93319
[7]	eval-rmse:1.91143
[8]	eval-rmse:1.90034
[9]	eval-rmse:1.89278
[10]	eval-rmse:1.88378
[11]	eval-rmse:1.87851
[12]	eval-rmse:1.87381
[13]	eval-rmse:1.86992
[14]	eval-rmse:1.86591
[15]	eval-rmse:1.86346
[16]	eval-rmse:1.86169
[17]	eval-rmse:1.85911
[18]	eval-rmse:1.85809
[19]	eval-rmse:1.85664
[20]	eval-rmse:1.85492
[21]	eval-rmse:1.85384
[22]	eval-rmse:1.85274
[23]	eval-rmse:1.85242
[24]	eval-rmse:1.85177
[25]	eval-rmse:1.84881
[26]	eval-rmse:1.84824
[27]	eval-rmse:1.84651
[28]	eval-rmse:1.84621
[29]	eval-rmse:1.84498
[30]	eval-rmse:1.84443
[31]	eval-rmse:1.84417
[32]	eval-rmse:1.84398
[33]	eval-rmse:1.84335
[34]	eval-rmse:1.84275
[35]	eval-rmse:1.84256
[36]	eval-rmse:1.84239
[37]	eval-rmse:1.84188
[38]	eval-rmse:1.84179
[39]	eval-rmse:1.84134
[40]	eval-rmse:1.84095
[41]	eval-rmse:1.84061
[42]	eval-rmse:1.84014
[43]	eval-rmse:1.8396

In [21]:
# Predict on the test set
y_pred = model.predict(test_data)

In [22]:
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
# Calculate RMSE and R2 score
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [24]:
print(f"RMSE: {rmse:.3f}")
print(f"R² Score: {r2:.3f}")

RMSE: 1.832
R² Score: 0.716


In [25]:
import os
from joblib import dump

In [26]:
# Directory where you want to save the model
model_dir = './savedModels/'

In [27]:
# Check if the directory exists, if not, create it
if not os.path.exists(model_dir):
    os.makedirs(model_dir)


In [28]:
# Save the trained model
dump(model, os.path.join(model_dir, 'model.joblib'))

['./savedModels/model.joblib']